In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [ ]:
X_train = pd.read_csv("/Users/b23/Desktop/ENS CHALLENGE/Catboost model/X_train_NHkHMNU.csv")
X_test = pd.read_csv("X_test_final.csv")
Y_train= pd.read_csv("/Users/b23/Desktop/ENS CHALLENGE/Catboost model/y_train_ZAN5mwg.csv")
Y_test = pd.read_csv("/Users/b23/Desktop/ENS CHALLENGE/Catboost model/y_test_random_final.csv")

In [ ]:
X_train = X_train.drop(['COUNTRY'], axis=1).fillna(0)
X_test = X_test.drop(['COUNTRY'], axis=1).fillna(0)

In [ ]:
Y_train = Y_train[Y_train.columns[0]]
Y_test = Y_test[Y_test.columns[0]]

In [ ]:
# Initialisation du StandardScaler
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(X_train)
x_test_scaled = scaler.transform(X_test)

# Définition du modèle
random_forest = RandomForestRegressor(random_state=42)

# Définition de l'espace des paramètres pour GridSearch
param_grid = {
    'n_estimators': [200],  # Nombre d'arbres dans la forêt
    'max_depth': [10],  # Profondeur maximale de l'arbre
    'min_samples_split': [10],  # Nombre minimum d'échantillons requis pour diviser un nœud interne
    'min_samples_leaf': [1],  # Nombre minimum d'échantillons requis pour être au niveau d'un nœud feuille
}

# Configuration de GridSearchCV
grid_search = GridSearchCV(estimator=random_forest, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_absolute_error')

# Entraînement avec GridSearchCV
grid_search.fit(x_train_scaled, Y_train)

# Meilleur modèle après GridSearch
best_model = grid_search.best_estimator_

# Prédiction avec le meilleur modèle
predictions = best_model.predict(x_test_scaled)

# Évaluation du modèle
mae = mean_absolute_error(Y_test, predictions)
mse = np.sqrt(mean_squared_error(Y_test, predictions))

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Root Mean Squared Error (RMSE): {mse}")
print(f"Meilleurs paramètres: {grid_search.best_params_}")

In [ ]:
# Accéder au meilleur modèle
best_model_rf = grid_search.best_estimator_

# Obtenir les importances des caractéristiques
feature_importances_gb = best_model_rf.feature_importances_

# Afficher les importances des caractéristiques
features = X_train.columns
importances_df_gb = pd.DataFrame({'Feature': features, 'Importance': feature_importances_gb})
importances_df_gb = importances_df_gb.sort_values(by='Importance', ascending=False)
print(importances_df_gb)